In [2]:
import time
import json
import pandas as pd
from web3 import Web3
import httpx

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
# pd.set_option('display.float_format', lambda x: '%.18f' % x)
print('started...')


thegraph_url = "https://api.thegraph.com/subgraphs/name/proy24/quickswap-polygon"

def get_uniswap_graphql(payload) -> pd.DataFrame:
    try:
        r = httpx.post(
            thegraph_url,
            data=json.dumps(payload),
            timeout=3600
        ).json()
        if r.get('data'):
            return pd.DataFrame(r['data']['pairs'])
        else:
            raise ValueError('shitty service gonna shit the bed again')
    except Exception as e:
        print(f"doh!: {e}")


graphql_payload_1st1000 = {
    "operationName":"pairs",
    "variables":{},
    "query":"query pairs {\n pairs(first: 1000) {\n id token0 {\n symbol\n name\n id\n decimals\n \n}\n token1 {\n symbol\n name\n id\n decimals\n \n}\n reserve0 reserve1 token0Price token1Price reserveUSD reserveETH trackedReserveETH volumeUSD untrackedVolumeUSD txCount \n} \n}\n"
}
graphql_payload_2nd1000 = {
    "operationName":"pairs",
    "variables":{},
    "query":"query pairs {\n pairs(first: 1000, skip:1000) {\n id token0 {\n symbol\n name\n id\n decimals\n \n}\n token1 {\n symbol\n name\n id\n decimals\n \n}\n reserve0 reserve1 token0Price token1Price reserveUSD reserveETH trackedReserveETH volumeUSD untrackedVolumeUSD txCount \n} \n}\n"
}
graphql_payload_3rd1000 = {
    "operationName":"pairs",
    "variables":{},
    "query":"query pairs {\n pairs(first: 1000, skip:2000) {\n id token0 {\n symbol\n name\n id\n decimals\n \n}\n token1 {\n symbol\n name\n id\n decimals\n \n}\n reserve0 reserve1 token0Price token1Price reserveUSD reserveETH trackedReserveETH volumeUSD untrackedVolumeUSD txCount \n} \n}\n"
}


df1 = get_uniswap_graphql(graphql_payload_1st1000)
assert len(df1.index) > 0, 'df1 empty'
df2 = get_uniswap_graphql(graphql_payload_2nd1000)
assert len(df2.index) > 0, 'df2 empty'
df3 = get_uniswap_graphql(graphql_payload_3rd1000)
assert len(df3.index) > 0, 'df3 empty'
df = pd.concat([df1, df2, df3])
print(f"df: {len(df)}")

# flatten thegraph token dict
df['symbols'] = [f"{x['symbol']}/{y['symbol']}" for x, y in zip(df['token0'], df['token1'])]
df['name0'] = [x['name'] for x in df['token0']]
df['name1'] = [x['name'] for x in df['token1']]
df['decimal0'] = [x['decimals'] for x in df['token0']]
df['decimal1'] = [x['decimals'] for x in df['token1']]
df['address0'] = [x['id'] for x in df['token0']]
df['address1'] = [x['id'] for x in df['token1']]

mung_cols = ['reserve0', 'reserve1', 'reserveETH', 'trackedReserveETH', 'volumeUSD', 'untrackedVolumeUSD', \
             'txCount', 'reserveUSD', 'token0Price', 'token1Price', 'decimal0', 'decimal1']
df[mung_cols] = df[mung_cols].apply(pd.to_numeric, errors='coerce')
df = df.rename(columns={'id':'pairAddr', 'token0Price': 'price0', 'token1Price': 'price1'})
df = df[['symbols', 'pairAddr', 'price0', 'price1', 'reserve0', 'reserve1', 'reserveETH', 'reserveUSD',\
         'trackedReserveETH', 'txCount', 'untrackedVolumeUSD', 'volumeUSD', 'decimal0', 'decimal1', 'address0', 'address1']]
df = df.sort_values('reserveUSD',  ascending=False)
df = df.reset_index()
del df['index']
df = df.copy()
#assert df.isnull().values.any(), 'NaN cells'
df.head(10)

started...
df: 3000


,symbols,pairAddr,price0,price1,reserve0,reserve1,reserveETH,reserveUSD,trackedReserveETH,txCount,untrackedVolumeUSD,volumeUSD,decimal0,decimal1,address0,address1
0,USDC/miMATIC,0x160532d2536175d65c03b97b0630a9802c274dad,0.991700,1.008369,8.399169e+06,8.469463e+06,4045.535993,1.679834e+07,4045.535994,802017,4.571194e+08,4.570801e+08,6,18,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,0xa3fa99a148fa48d14ed51d610c367c61876997f1
1,WMATIC/QUICK,0x019ba0325f1988213d448b3472fa1cf8d07618d7,243.771603,0.004102,4.511018e+06,1.850510e+04,3637.156280,1.510220e+07,3637.156280,4015859,1.135818e+09,1.135818e+09,18,18,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,0x831753dd7087cac61ab5644b308642cc1c33dc13
2,WETH/QUICK,0x1bd06b96dd42ada85fdd0795f3b4a79db914add5,0.098358,10.166986,1.160485e+03,1.179863e+04,2320.350438,9.634560e+06,2320.350438,2413779,6.444822e+08,6.444822e+08,18,18,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619,0x831753dd7087cac61ab5644b308642cc1c33dc13
3,USDC/GHST,0x096c5ccb33cfc5732bcd1f3195c13dbefc4c82f4,1.887829,0.529709,3.024324e+06,1.602012e+06,1461.506227,6.068472e+06,1456.731770,367496,2.378322e+08,2.378534e+08,6,18,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,0x385eeac5cb85a38a9a07a70c73e0a3271cfb54a7
4,USDC/QUICK,0x1f1e4c845183ef6d50e9609f16f6f9cae43bc9cb,407.070104,0.002457,2.307571e+06,5.668732e+03,1112.992411,4.621374e+06,1112.992411,2142838,1.081548e+09,1.081548e+09,6,18,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,0x831753dd7087cac61ab5644b308642cc1c33dc13
5,NEXO/WETH,0x10062ec62c0be26cc9e2f50a1cf784a89ded075f,1539.525118,0.000650,6.555790e+05,4.258320e+02,851.662522,3.533491e+06,851.663967,37449,9.654010e+06,9.651758e+06,18,18,0x41b3966b4ff7b427969ddf5da3627d6aeae9a48e,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619
6,GFARM2/DAI,0x0c7ad41d3e0dbc1cfdcdd717afb0a72a65cdf069,0.001640,609.740613,1.448947e+03,8.834815e+05,427.950928,1.776941e+06,426.108702,40728,3.306061e+07,3.301471e+07,18,18,0x7075cab6bcca06613e2d071bd918d1a0241379e2,0x8f3cf7ad23cd3cadbd9735aff958023239c6a063
7,UCO/WETH,0x25bae75f6760ac30554cc62f9282307c3038c3a0,15952.135065,0.000063,2.258481e+06,1.415786e+02,283.157201,1.174296e+06,283.157201,578,2.687269e+05,2.676818e+05,18,18,0x3c720206bfacb2d16fa3ac0ed87d2048dbc401fc,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619
8,TECH/WETH,0x204a7adc76db7fe8c5e5f499cb3c4cff6d7282c2,69392.044470,0.000014,8.883457e+06,1.280184e+02,256.082490,1.064116e+06,256.036766,19528,4.203815e+11,5.983447e+06,18,18,0x6286a9e6f7e745a6d884561d88f94542d6715698,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619
9,USDC/MASK,0x253d637068fbf11b18d0f2a1bf3b167d37802687,9.488262,0.105393,5.227030e+05,5.508943e+04,251.674066,1.045406e+06,251.674066,16346,5.781570e+06,5.780554e+06,6,18,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,0x2b9e7ccdf0f4e5b24757c1e1a80e311e34cb10c7


In [3]:
df.tail()

,symbols,pairAddr,price0,price1,reserve0,reserve1,reserveETH,reserveUSD,trackedReserveETH,txCount,untrackedVolumeUSD,volumeUSD,decimal0,decimal1,address0,address1
2995,WMATIC/BALL,0x1d579674616af54595b33feb6ad9d24f550ce20a,1.000000e-04,1.000000e+04,0.000000e+00,1.000000e-13,0.0,0.0,0.0,2,0.0,0.0,18,18,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,0xeb131eddb1e8334ed8ac9f2e4a149076fdf543e1
2996,WMATIC/TES,0x1d597b609fd91da75a8fd6f57225f1adcb85cec6,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0,0.0,0.0,18,9,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,0x971e130c2236b2e5965501a2e2a2b43109a0c3b1
2997,BALL/BURN,0x1d6045c58d21937f135e878c84f89f896ebb3418,1.973739e+03,5.066526e-04,4.530000e-14,0.000000e+00,0.0,0.0,0.0,2,0.0,0.0,18,18,0x00c39b2930fe947ac8e7cd4a514ae9738b989196,0x41859e7de5b4ca7fda4ba3da4ef93a5a21d3d7f7
2998,CHI/CZR,0x1d68a332f46481276b82ee00b3390b8cbaf856fa,3.993900e-12,2.503756e+11,2.000000e+00,5.007511e+11,0.0,0.0,0.0,2,0.0,0.0,0,18,0x0000000000004946c0e9f43f4dee607b0ef1fa1c,0xd422089785ada2762343b209b43bc48dd7ad4ae9
2999,WMATIC/PLYTF,0x1e7d0b49c5c00f56ef0a9c82d811c2b4d0299bd9,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0,0.0,0.0,18,18,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,0xb848d02d94fe22d4e3c58368ebf8e22b0393e98d
